## Web scrape hemköp with Selenium

In [2]:
import os

import pandas as pd
import numpy as np
import requests
import spoonacular as sp

import funcs
import importlib
importlib.reload(funcs)

<module 'funcs' from '/Users/gavinnguyen/Documents/NOD Bootcamp/Project - Web scrape_API (End of week 5 start)/funcs.py'>

#### Scrape disabled

In [209]:
# # scraper.py
# from scraper import scraper
#
# # scraper
# df = scraper()

Lengths: 298, 298, 298, 298, 298, 298, 298


#### Saved scrape

In [225]:
# saved dataframe so we don't have to repeatedly scrape for testing
df = pd.read_csv('saved_data_2.csv')
df.drop('Unnamed: 0', axis=1, inplace=True)

## Data Cleaning

In [226]:
df

,Name,Price,Original Price,Original compare price,Discounted compare price,Brand,Weight
0,Bacon Rökt&skivat,10.00,14.95,"106,79 kr/kg","71,43 kr/kg",GARANT,140g
1,Kyckling Filé Fryst,59.00,69.95,"77,72 kr/kg","65,55 kr/kg",ELDORADO,900g
2,Levain Vete,25.00,39.95,"53,27 kr/kg","33,33 kr/kg",SÖDERVIDINGE,750g
3,Normalsaltat Smör & Rapsolja 75%,49.00,59.95,"99,92 kr/kg","81,67 kr/kg",BREGOTT,600g
4,Druvor Gröna Klass 1,20.00,39.95,"79,90 kr/kg","40,00 kr/kg",NaN,NaN
...,...,...,...,...,...,...,...
292,Vegetarisk Bredbar Pastej av Ärtprotein,18.95,NaN,"94,75 kr/kg",NaN,PASTEJKÖKET,200g
293,Citron Såpa Allrengöring,22.50,NaN,"30,00 kr/l",NaN,GRUMME,750ml
294,Krögarpytt Klassisk Fryst,48.95,NaN,"67,99 kr/kg",NaN,FELIX,720g
295,Fiber Formbröd Glutenfri Fryst,36.95,NaN,"73,90 kr/kg",NaN,FRIA,500g


In [227]:
df['Original compare price'] = df['Original compare price'].str.replace('Jmf pris ', '')
df['Discounted compare price'] = df['Discounted compare price'].str.replace('Jmf pris ', '')
df['Original Price'] = df['Original Price'].str.replace('Ord pris ', '')

AttributeError: Can only use .str accessor with string values!

### Changing dtypes

In [228]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 297 entries, 0 to 296
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Name                      297 non-null    object 
 1   Price                     297 non-null    float64
 2   Original Price            50 non-null     float64
 3   Original compare price    295 non-null    object 
 4   Discounted compare price  229 non-null    object 
 5   Brand                     280 non-null    object 
 6   Weight                    280 non-null    object 
dtypes: float64(2), object(5)
memory usage: 16.4+ KB


#### Changing Price and original price to float after some cleaning

In [229]:
# todo:         FUTURE IMPLEMENTATION: split kr/kg and kr/l into separate columns
df.Price = df['Price'].str.replace(',', '.')
df['Original Price'] = df['Original Price'].str.replace(' kr', '')
df['Original Price'] = df['Original Price'].str.replace(',', '.')
df['Original Price'] = df['Original Price'].str.replace('/kg', '')

AttributeError: Can only use .str accessor with string values!

In [230]:
df.Price = df['Price'].astype('float')
df['Original Price'] = df['Original Price'].astype(float)

In [231]:
df.fillna(np.NaN, inplace=True)

### Translate the ingredients
Swedish to english using google translate API

In [232]:
translated_names = funcs.translate_food_names(df)

ERROR1:retrying... Bacon Rökt&skivat
ERROR2:retrying... Bacon Rökt&skivat
ERROR3:retrying... Bacon Rökt&skivat
ERROR4:retrying... Bacon Rökt&skivat
ERROR5:retrying... Bacon Rökt&skivat


KeyboardInterrupt: 

In [ ]:
# todo:         LIMITATION: translate errors. e.g. Diskborste -> disk brush

In [9]:
# translated_names = pd.read_csv('translated_names_2.csv')

In [233]:
translated_names.drop('Unnamed: 0', axis=1, inplace=True)

KeyError: "['Unnamed: 0'] not found in axis"

## Recipes API - spoonacular API (input ingredients, output recipes

Check recipes based on ingredients

https://spoonacular.com/food-api

In [234]:
# Reading spoonacular ingredients list csv
accepted_ingredients = pd.read_csv('ingredients.csv')

### Data Cleaning

In [235]:
# Split all rows at ;, and expand=True means split into different columns
# Assign to new column names
accepted_ingredients[['Ingredient', 'Ingredient ID']] = accepted_ingredients['ingredient;ingredientId'].str.split(';', expand=True)

In [236]:
# Dropping old column
accepted_ingredients.drop('ingredient;ingredientId', axis=1, inplace=True)
# Renaming column
translated_names.rename(columns={'0':'Ingredient'}, inplace=True)
# Lower case everything for case insensitivity
translated_names = translated_names.Ingredient.str.lower()
# Convert into DataFrame to access some df methods later on
translated_names = pd.DataFrame(translated_names)

In [237]:
# Check for accepted ingredients
parsed_ingredients = funcs.parse_ingredients(translated_names, accepted_ingredients)

In [238]:
len(parsed_ingredients)
# todo:         LIMITATION: e.g. bacon will be extracted from bacon-snacks

190

### Input parsed_ingredients into Spoonacular recipes API to get a list of recipes

In [18]:
api_key = '43f7682fd91f42c0b2b3c32a83f6f634'

In [21]:
# spoonacular input is ingredient1,ingredient2,ingredient3...
ingredients = ','.join(parsed_ingredients)

In [63]:
ingredients

'salt,butter,sugar,water,flour,garlic,eggs,onion,milk,pepper,egg,honey,cream cheese,chicken,sour cream,vanilla,sea salt,powdered sugar,bacon,tomatoes,oil,chicken broth,chocolate,beef,zucchini,strawberries,potatoes,fish,tomato,avocado,lemon,shrimp,mushrooms,blueberries,bread,cucumber,banana,coconut,pasta,apple,peas,rice,broccoli,cream,basil,cooking oil,ham,lime,ice,coconut milk,mango,mint,cookies,whipped cream,vanilla ice cream,olives,salted butter,pumpkin,beer,corn,potato,yogurt,salmon,fruit,cheese,dark chocolate,ice cream,black beans,arugula,spaghetti,tofu,prosciutto,tortilla chips,sausage,cocoa,pesto,chili sauce,berries,baguette,pear,watermelon,tomato puree,mushroom,chili,parmigiano reggiano,cheddar,vegan cheese,juice,almond,herbs,meat,mozzarella,pizza dough,egg noodles,pumpkin seeds,sage,tea,lemongrass,saffron,sunflower oil,noodles,chorizo,couscous,green olives,tomato soup,cookie,gin,stock,broth,steak,grapes,strawberry,seeds,salmon fillet,popcorn,vegetable,jam,tuna,vanilla yogurt,ta

In [28]:
url = f'https://api.spoonacular.com/recipes/findByIngredients?apiKey={api_key}&ingredients={ingredients}'

In [29]:
spoonacular = requests.get(url)

In [31]:
spoonacular

<Response [200]>

In [35]:
recipes_df = pd.DataFrame(spoonacular.json()).sort_values(by='likes', ascending=False)

### Get ingredient list for each recipe

In [187]:
recipes_df

,id,title,image,imageType,usedIngredientCount,missedIngredientCount,missedIngredients,usedIngredients,unusedIngredients,likes
9,715560,World’s Greatest Lasagna Roll Ups,https://spoonacular.com/recipeImages/715560-31...,jpg,10,4,"[{'id': 11549, 'amount': 8.0, 'unit': 'oz', 'u...","[{'id': 1123, 'amount': 1.0, 'unit': '', 'unit...","[{'id': 1001, 'amount': 1.0, 'unit': 'serving'...",32767
7,664087,Turkish Baharat Meatballs with Lentil Pilaf,https://spoonacular.com/recipeImages/664087-31...,jpg,12,7,"[{'id': 1002014, 'amount': 0.5, 'unit': 'tsp',...","[{'id': 10123, 'amount': 1.0, 'unit': '', 'uni...","[{'id': 1001, 'amount': 1.0, 'unit': 'serving'...",7
0,631747,Dutch Oven Paella,https://spoonacular.com/recipeImages/631747-31...,jpg,13,3,"[{'id': 14106, 'amount': 0.33333334, 'unit': '...","[{'id': 15270, 'amount': 1.0, 'unit': 'lb', 'u...","[{'id': 1001, 'amount': 1.0, 'unit': 'serving'...",6
4,982376,Chicken Noodle Casserole Dish,https://spoonacular.com/recipeImages/982376-31...,jpg,12,3,"[{'id': 11124, 'amount': 1.0, 'unit': 'c', 'un...","[{'id': 10028033, 'amount': 0.5, 'unit': 'c', ...","[{'id': 11215, 'amount': 1.0, 'unit': 'serving...",5
3,654327,Paella for Four; A Wonderful Spanish Mixed Sea...,https://spoonacular.com/recipeImages/654327-31...,jpg,12,2,"[{'id': 2027, 'amount': 2.0, 'unit': 't', 'uni...","[{'id': 10211821, 'amount': 1.0, 'unit': 'T', ...","[{'id': 1001, 'amount': 1.0, 'unit': 'serving'...",3
5,652134,Mixed Paella,https://spoonacular.com/recipeImages/652134-31...,jpg,12,5,"[{'id': 2004, 'amount': 1.0, 'unit': '', 'unit...","[{'id': 6194, 'amount': 1.0, 'unit': 'quart', ...","[{'id': 1001, 'amount': 1.0, 'unit': 'serving'...",2
8,1054737,How to make the World’s Greatest Lasagna Roll Ups,https://spoonacular.com/recipeImages/1054737-3...,jpg,11,4,"[{'id': 11549, 'amount': 16.0, 'unit': 'oz', '...","[{'id': 1123, 'amount': 1.0, 'unit': '', 'unit...","[{'id': 1001, 'amount': 1.0, 'unit': 'serving'...",2
1,634582,Beef and Vegetable Penne Casserole,https://spoonacular.com/recipeImages/634582-31...,jpg,13,4,"[{'id': 2025, 'amount': 1.0, 'unit': 'teaspoon...","[{'id': 2044, 'amount': 15.0, 'unit': 'leaves'...","[{'id': 1001, 'amount': 1.0, 'unit': 'serving'...",1
2,651342,Meatball Subs,https://spoonacular.com/recipeImages/651342-31...,jpg,13,4,"[{'id': 1032009, 'amount': 0.5, 'unit': 'teasp...","[{'id': 10023567, 'amount': 1.5, 'unit': 'poun...","[{'id': 1001, 'amount': 1.0, 'unit': 'serving'...",1
6,651965,Mini Meatball & Ravioli Soup With Garlic Croutons,https://spoonacular.com/recipeImages/651965-31...,jpg,12,5,"[{'id': 2004, 'amount': 1.0, 'unit': '', 'unit...","[{'id': 23567, 'amount': 1.0, 'unit': 'pound',...","[{'id': 1077, 'amount': 1.0, 'unit': 'serving'...",1


In [212]:
print('Used ingredients list:\n')
for i, item in enumerate(recipes_df.usedIngredients):
    print(recipes_df.usedIngredients[9][i]['original'])

Used ingredients list:

1 egg
1 (15 oz) can diced tomatoes (I use flavored fire roasted with basil)
2 cloves garlic (minced)
1 pound Italian Sausage
Uncooked Lasagna noodles
1 pound Lean Ground Beef (I like 93/7)
sliced mozzarella cheese
½ cup chopped onion
¾ cup grated parmesan reggiano
(16 oz) ricotta cheese


### Fetch recipes using recipe ID

In [88]:
# Recipe IDs separated by commas
recipe_ids = ','.join([str(x) for x in recipes_df.id.to_list()])

In [91]:
url_recipes = f'https://api.spoonacular.com/recipes/informationBulk?apiKey={api_key}&ids={recipe_ids}'

In [92]:
spoonacular_recipes = requests.get(url_recipes)

In [154]:
recipe_list = pd.DataFrame(spoonacular_recipes.json())

In [191]:
required_ingredients = []
required_ingredients_amount = []
for i, e in enumerate(recipe_list.extendedIngredients):
    required_ingredients.append(recipe_list.extendedIngredients[0][i]['name'])
    required_ingredients_amount.append(recipe_list.extendedIngredients[0][i]['amount'])
    required_ingredients

In [ ]:
# todo: match with deal data to get x amount to buy, price

## Desired output
- ✅Output recipes, with what ingredients you need to get and how many

- Display discounted price
- Display original prices

- Display leftover ingredients
- (optional) display price of leftover ingredients

- Subtract that with the used ingredients and get the actual price for the recipe

In [181]:
    df['Name'] = translated_names

In [184]:
df.drop(columns='name', inplace=True)

In [239]:
df

,Name,Price,Original Price,Original compare price,Discounted compare price,Brand,Weight
0,Bacon Rökt&skivat,10.00,14.95,"106,79 kr/kg","71,43 kr/kg",GARANT,140g
1,Kyckling Filé Fryst,59.00,69.95,"77,72 kr/kg","65,55 kr/kg",ELDORADO,900g
2,Levain Vete,25.00,39.95,"53,27 kr/kg","33,33 kr/kg",SÖDERVIDINGE,750g
3,Normalsaltat Smör & Rapsolja 75%,49.00,59.95,"99,92 kr/kg","81,67 kr/kg",BREGOTT,600g
4,Druvor Gröna Klass 1,20.00,39.95,"79,90 kr/kg","40,00 kr/kg",NaN,NaN
...,...,...,...,...,...,...,...
292,Vegetarisk Bredbar Pastej av Ärtprotein,18.95,NaN,"94,75 kr/kg",NaN,PASTEJKÖKET,200g
293,Citron Såpa Allrengöring,22.50,NaN,"30,00 kr/l",NaN,GRUMME,750ml
294,Krögarpytt Klassisk Fryst,48.95,NaN,"67,99 kr/kg",NaN,FELIX,720g
295,Fiber Formbröd Glutenfri Fryst,36.95,NaN,"73,90 kr/kg",NaN,FRIA,500g


In [198]:
required_ingredients_df = pd.DataFrame(required_ingredients, required_ingredients_amount)

In [201]:
required_ingredients_df.reset_index()

,index,0
0,8.00,tomato sauce
1,16.00,tomato sauce
2,1.50,basil
3,1.00,egg
4,1.00,extra virgin olive oil
5,15.00,canned tomatoes
6,4.00,parsley
7,2.00,garlic
8,0.25,ground pepper
9,1.00,sausage


In [203]:
required_ingredients_df

,0
8.00,tomato sauce
16.00,tomato sauce
1.50,basil
1.00,egg
1.00,extra virgin olive oil
15.00,canned tomatoes
4.00,parsley
2.00,garlic
0.25,ground pepper
1.00,sausage


In [242]:
df['Name'] = translated_names

In [264]:
df.loc[df.Name.str.contains('ricotta')]

,Name,Price,Original Price,Original compare price,Discounted compare price,Brand,Weight
119,ricotta spinaci tortelloni fresh pasta,29.95,NaN,"119,80 kr/kg","110,00 kr/kg",GARANT,250g


In [ ]:
264, 156, 127, 254, 290, 103, 35, 119

In [272]:
df_ingredients = df.iloc[[264, 156, 127, 254, 290, 103, 35, 119]]

In [273]:
df_ingredients.loc['Total'] = df_ingredients.sum(numeric_only=True)

/var/folders/dy/x6z2mb456s9dvq77dv75k9xr0000gn/T/ipykernel_60183/171260471.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ingredients.loc['Total'] = df_ingredients.sum(numeric_only=True)


In [275]:
df_ingredients

,Name,Price,Original Price,Original compare price,Discounted compare price,Brand,Weight
264,egg 15p free indoor medium,41.95,NaN,"2,80 kr/st",NaN,GARANT,15p
156,crushed tomatoes herbs,12.50,NaN,"32,05 kr/kg","25,64 kr/kg",GARANT,390g
127,garlic salami sliced,18.95,NaN,"236,88 kr/kg","200,00 kr/kg",GARANT,80g
254,whole grain lasagna 1-3 years,17.50,NaN,"79,55 kr/kg","68,18 kr/kg",NESTLÉ,220 g
290,beef mince 12% sweden,69.95,NaN,"139,90 kr/kg",NaN,GARANT EKO,500g
103,mozzarella,14.95,NaN,"119,60 kr/kg","104,00 kr/kg",FRESCA D´ORO,125g
35,onion yellow bundle class 1,10.00,13.95,NaN,NaN,NaN,NaN
119,ricotta spinaci tortelloni fresh pasta,29.95,NaN,"119,80 kr/kg","110,00 kr/kg",GARANT,250g
Total,NaN,215.75,13.95,NaN,NaN,NaN,NaN


In [276]:
df

,Name,Price,Original Price,Original compare price,Discounted compare price,Brand,Weight
0,bacon smoked sliced,10.00,14.95,"106,79 kr/kg","71,43 kr/kg",GARANT,140g
1,chicken fillet frozen,59.00,69.95,"77,72 kr/kg","65,55 kr/kg",ELDORADO,900g
2,levain wheat,25.00,39.95,"53,27 kr/kg","33,33 kr/kg",SÖDERVIDINGE,750g
3,normal salted butter & rapeseed oil 75%,49.00,59.95,"99,92 kr/kg","81,67 kr/kg",BREGOTT,600g
4,grapes green class 1,20.00,39.95,"79,90 kr/kg","40,00 kr/kg",NaN,NaN
...,...,...,...,...,...,...,...
293,lemon soap all cleaning,22.50,NaN,"30,00 kr/l",NaN,GRUMME,750ml
294,crows of classic frozen,48.95,NaN,"67,99 kr/kg",NaN,FELIX,720g
295,fiber shape bread gluten -free frozen,36.95,NaN,"73,90 kr/kg",NaN,FRIA,500g
296,falu rågrut original,13.95,NaN,"59,36 kr/kg",NaN,WASA,235g
